# Pipeline Risk Assessment: Muhlbauer Definitive Approach Baseline

This repository implements a **Quantitative Risk Assessment (QRA)** engine based on the Muhlbauer Definitive Approach (www.pipelinerisk.net). It rejects indexing and scoring in favor of physical calculations using the **Exposure-Mitigation-Resistance** triad. It can also be used to develop a fully probabilistic risk assessment (PRA)

## 🎯 The Philosophy of Conservatism
This model does not use "safety multipliers." Instead, it requires the Risk Assessor to select input values based on the intended **Confidence Level**:

1. **P99+ (Integrity Management):** Use the "High Path" (Upper bound of threats, lower bound of resistance). Appropriate for public safety and regulatory compliance.
2. **P50 (Strategic Planning/Due Diligence):** Use the "Central Tendency" (Mean/Median). Appropriate for asset valuation and M&A due diligence.

## 🏗 Mathematical Framework
### Frequency of Failure (FoF)
$$FoF = \text{Exposure} \times (1 - \text{Mitigation}) \times (1 - \text{Resistance})$$
$$TTF = {Resistance} / (Exposure \times (1-Mitigation))$$
**Time-Dependent Threats:** Modeled via a **Weibull Distribution** or equivalent to determine the annual frequency of reaching a defined Limit State (Critical Wall Thickness).

### Consequence of Failure (CoF)
Calculated as the consequence potential based on the physical **Hazard Zone** that could result from a pipeline release:
$$\text{CoF (\$)} = \text{Spill } (ft^3) \times \text{Spread } (ft^2/ft^3) \times \text{Receptor } (Units/ft^2) \times \text{Product } (\$/Unit)$$

## 🚀 Getting Started
1. Open `PL_risk_Muhlbauer.ipynb` in a Jupyter environment.
2. Complete the **Conservatism Strategy Declaration**.
3. Input segment-specific data.
4. Run the engine to generate the Audit Log and Risk Profile.

In [15]:
import math
import warnings
import sys

# Custom warning format to remove file path and line number
def custom_formatwarning(message, category, filename, lineno, line=None):
    return f"{category.__name__}: {message}\n"
warnings.formatwarning = custom_formatwarning

class MuhlbauerRiskEngine:
    """
    Baseline Pipeline Risk Engine (2026 Edition)
    Logic: Exposure-Mitigation-Resistance (EMR)
    Consequence: Dimensional Hazard Zone
    """
    def __init__(self, segment_name, diameter_in, wall_thk_in, smys_psi, pressure_psi, intent="Integrity"):
        self.name = segment_name
        self.d = diameter_in
        self.t_initial = wall_thk_in
        self.smys = smys_psi
        self.p = pressure_psi
        self.intent = intent  # "Integrity" (P95+) or "Strategic" (P50)
        self.audit_log = {}

    def log_input(self, category, value, percentile, justification):
        self.audit_log[category] = {"value": value, "percentile": percentile, "justification": justification}
        if self.intent == "Integrity" and "P50" in percentile:
            warnings.warn(f"Strategy Conflict: Using P50 for {category} in an Integrity assessment may underestimate risk.")

    def calculate_corrosion_ttf_and_fof(self, age_yrs, corrosion_rate_ipy, mitigation_eff, beta=4.0):
        """
        Calculates Time to Failure (TTF) and Annual Frequency (FoF).
        TTF = Resistance / (Exposure * (1 - Mitigation))
        """
        # 1. Calculate 'Resistance' (The budget of wall thickness before failure)
        t_crit = (self.p * self.d) / (2 * self.smys)
        resistance_remaining = self.t_initial - t_crit

        # 2. Calculate 'Net Exposure' (Exposure * (1 - Mitigation))
        net_exposure = corrosion_rate_ipy * (1 - mitigation_eff)

        # 3. Calculate TTF (The deterministic life span)
        if net_exposure > 0:
            ttf_total = resistance_remaining / net_exposure
            ttf_remaining = ttf_total - age_yrs

            # 4. Convert to Probabilistic FoF using Weibull
            eta = ttf_total # Characteristic life
            fof_at_age = 0.0 # Default value

            if eta <= 0: # If total life is non-positive, consider failure certain if pipe has age
                if age_yrs > 0:
                    fof_at_age = 1.0
                # else fof_at_age remains 0.0 if age_yrs is 0
            else:
                try:
                    fof_at_age = 1 - math.exp(-(age_yrs / eta)**beta)
                except OverflowError:
                    fof_at_age = 0.99 # Set to 1.0 if the exponent is too large

            # Apply user-requested override for TTF <= 0.0 for reporting
            if ttf_remaining <= 0.0:
                warnings.warn(f"[{self.name}] TTF Remaining calculated as {ttf_remaining:.1f} years. Adjusting to 0.1 years for display/reporting. Note: A TTF of 0.0 years often implies insufficient recent inspection data. Consider updating inspection records or taking credit for non-leaking segments (Barlow formula at NOP).")
                ttf_remaining = 0.1

        else: # net_exposure <= 0 (e.g., zero corrosion rate or 100% effective mitigation)
            ttf_remaining = float('inf')
            fof_at_age = 0.0

        return max(ttf_remaining, 0.0), max(fof_at_age, 0.0)

    def calculate_fof_third_party(self, activity_km_yr, general_mitigation_eff, tp_mitigation_eff, resistance_prob):
        """Random Threat logic with two mitigation factors"""
        return activity_km_yr * (1 - general_mitigation_eff) * (1 - tp_mitigation_eff) * (1 - resistance_prob)

    def calculate_hazard_zone_cof(self, spill_vol_ft3, spread_factor, cost_per_unit, receptor_density):
        """Physical footprint calculation in USD"""
        impact_area_ft2 = spill_vol_ft3 * spread_factor
        total_receptors = impact_area_ft2 * receptor_density
        return total_receptors * cost_per_unit

# --- SIMULATION ---
pipe = MuhlbauerRiskEngine("Baseline_Seg_001", 20.0, 0.375, 60000, 900, intent="Integrity")

# Scenario: 40 year old pipe, 0.010 ipy corrosion, 80% CP mitigation
ttf_rem, p_corr = pipe.calculate_corrosion_ttf_and_fof(40, 0.010, 0.80)
p_tp = pipe.calculate_fof_third_party(0.5, 0.90, 0.85, 0.98) # Added a second mitigation factor

# Combined Risk
total_fof = 1 - ((1 - p_tp) * (1 - p_corr))
cof = pipe.calculate_hazard_zone_cof(10000, 0.25, 200000, 0.001)

print(f"--- {pipe.name} RESULTS ---")
print(f"Time to Failure Remaining: {ttf_rem:.1f} years")
print(f"Annual Frequency (FoF): {total_fof:.6f}")
print(f"CoF Value (CoF):  ${cof:,.2f}")
print(f"Total Annual Risk:        ${total_fof * cof:,.2f}")

--- Baseline_Seg_001 RESULTS ---
Time to Failure Remaining: 72.5 years
Annual Frequency (FoF): 0.016003
CoF Value (CoF):  $500,000.00
Total Annual Risk:        $8,001.27


# 📋 Risk Assessment Audit & Conservatism Protocol

**Assessor Name:** ___________________  
**Assessment Date:** 2026-XX-XX  
**Intent of Results:** [ ] Integrity Management (P95+) | [ ] Strategic/Valuation (P50)

### Input Selection Guidance
| Variable | P50 (Strategic) | P95+ (Integrity) |
| :--- | :--- | :--- |
| **Corrosion Rate** | Median ILI Growth Rate | Max Observed Pit Growth |
| **Excavation Activity**| Yearly Average | Peak Monthly (Extrapolated) |
| **Mitigation Eff.** | Nominal System Performance | Worst-Case Downtime Included |
| **Wall Thickness** | Nominal Thickness | Nominal minus Mfg. Tolerance |

### Justification Log
*Every input must be rationalized. If using P50 for an Integrity assessment, provide a technical deviation note below.*
- **Corrosion Justification:**
- **Third-Party Justification:**
- **Consequence Justification:**

## 📊 Running the Engine with Sample Data

Let's use the provided sample data to simulate risk assessments for multiple pipeline segments. We will load the data into a pandas DataFrame and then iterate through each row to apply the `MuhlbauerRiskEngine`.

In [16]:
import pandas as pd
import io

# Sample data provided by the user
data = """Segment_ID,Dia (in),Wall (in),SMYS (psi),Press (psi),Age,Ext Corr Rate,Coat/CP Eff,TP Act,TP Mit,Res Prob
Seg_001_Urban,24.0,0.500,65000,1000,15,0.008,0.95,2.5,0.85,0.99
Seg_002_Rural,12.0,0.250,52000,800,45,0.012,0.70,0.1,0.90,0.95
Seg_003_HCA,30.0,0.625,70000,1200,10,0.005,0.98,5.0,0.75,0.99
Seg_004_Vintage,16.0,0.250,42000,600,60,0.020,0.50,0.3,0.92,0.90
"""

df_segments = pd.read_csv(io.StringIO(data))
display(df_segments)

,Segment_ID,Dia (in),Wall (in),SMYS (psi),Press (psi),Age,Ext Corr Rate,Coat/CP Eff,TP Act,TP Mit,Res Prob
0,Seg_001_Urban,24.0,0.500,65000,1000,15,0.008,0.95,2.5,0.85,0.99
1,Seg_002_Rural,12.0,0.250,52000,800,45,0.012,0.70,0.1,0.90,0.95
2,Seg_003_HCA,30.0,0.625,70000,1200,10,0.005,0.98,5.0,0.75,0.99
3,Seg_004_Vintage,16.0,0.250,42000,600,60,0.020,0.50,0.3,0.92,0.90


In [17]:
# Assuming MuhlbauerRiskEngine class is defined in an earlier cell

print("--- MULTI-SEGMENT RISK ASSESSMENT ---")

results = []

for index, row in df_segments.iterrows():
    segment_name = row['Segment_ID']
    diameter = row['Dia (in)']
    wall_thickness = row['Wall (in)']
    smys = row['SMYS (psi)']
    pressure = row['Press (psi)']
    age = row['Age']
    ext_corrosion_rate = row['Ext Corr Rate'] # Changed column name
    coat_cp_efficiency = row['Coat/CP Eff'] # Changed column name
    tp_activity = row['TP Act']
    tp_mitigation_eff = row['TP Mit'] # New column
    tp_resistance_prob = row['Res Prob']

    # Instantiate the engine for each segment
    # For this example, we'll assume 'Integrity' intent for all
    pipe = MuhlbauerRiskEngine(segment_name, diameter, wall_thickness, smys, pressure, intent="Integrity")

    # Calculate corrosion-related FoF and TTF
    ttf_rem, f_corr = pipe.calculate_corrosion_ttf_and_fof(age, ext_corrosion_rate, coat_cp_efficiency)

    # Calculate third-party FoF, passing both general and specific TP mitigation
    f_tp = pipe.calculate_fof_third_party(tp_activity, coat_cp_efficiency, tp_mitigation_eff, tp_resistance_prob)

    # Combined FoF
    total_fof = 1 - ((1 - f_tp) * (1 - f_corr))

    # For CoF, we'll use example fixed values for now, as specific CoF inputs are not in the sample data
    # In a real scenario, these would come from segment-specific data as well.
    example_spill_vol_ft3 = 10000
    example_spread_factor = 0.25
    example_cost_per_unit = 20000
    example_receptor_density = 0.001
    cof = pipe.calculate_hazard_zone_cof(example_spill_vol_ft3, example_spread_factor, example_cost_per_unit, example_receptor_density)

    # Total Annual Risk
    total_annual_risk = total_fof * cof

    print(f"\n--- {pipe.name} RESULTS ---")
    print(f"Time to Failure Remaining: {ttf_rem:.1f} years")
    print(f"Annual Frequency (FoF): {total_fof:.6f}")
    print(f"CoF Value (CoF):  ${cof:,.2f}")
    print(f"Total Annual Risk:        ${total_annual_risk:,.2f}")

    results.append({
        'Segment_ID': segment_name,
        'TTF_Remaining_Years': ttf_rem,
        'Annual_FoF': total_fof,
        'CoF_USD': cof,
        'Total_Annual_Risk_USD': total_annual_risk
    })

df_results = pd.DataFrame(results)
print("\n--- SUMMARY OF ALL SEGMENTS ---")
# Apply comma formatting to currency columns
formatted_df_results = df_results.copy()
formatted_df_results['CoF_USD'] = formatted_df_results['CoF_USD'].apply(lambda x: f'${x:,.2f}')
formatted_df_results['Total_Annual_Risk_USD'] = formatted_df_results['Total_Annual_Risk_USD'].apply(lambda x: f'${x:,.2f}')
display(formatted_df_results.round(2))

--- MULTI-SEGMENT RISK ASSESSMENT ---

--- Seg_001_Urban RESULTS ---
Time to Failure Remaining: 773.5 years
Annual Frequency (FoF): 0.000188
CoF Value (CoF):  $50,000.00
Total Annual Risk:        $9.38

--- Seg_002_Rural RESULTS ---
Time to Failure Remaining: 0.1 years
Annual Frequency (FoF): 0.671749
CoF Value (CoF):  $50,000.00
Total Annual Risk:        $33,587.47

--- Seg_003_HCA RESULTS ---
Time to Failure Remaining: 3668.6 years
Annual Frequency (FoF): 0.000250
CoF Value (CoF):  $50,000.00
Total Annual Risk:        $12.50

--- Seg_004_Vintage RESULTS ---
Time to Failure Remaining: 0.1 years
Annual Frequency (FoF): 1.000000
CoF Value (CoF):  $50,000.00
Total Annual Risk:        $50,000.00

--- SUMMARY OF ALL SEGMENTS ---


,Segment_ID,TTF_Remaining_Years,Annual_FoF,CoF_USD,Total_Annual_Risk_USD
0,Seg_001_Urban,773.46,0.00,"$50,000.00",$9.38
1,Seg_002_Rural,0.10,0.67,"$50,000.00","$33,587.47"
2,Seg_003_HCA,3668.57,0.00,"$50,000.00",$12.50
3,Seg_004_Vintage,0.10,1.00,"$50,000.00","$50,000.00"
